<a href="https://colab.research.google.com/github/ambgeo/Youtube/blob/main/car_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Detecção de Carros utilizando o Pacote GeoAI

* Fonte: GeoAI - https://geoai.gishub.org

In [ ]:
##Instalação do pacote geoai
!pip install geoai-py

## IMporte a biblioteca

In [ ]:
import geoai

## Baixe o dado para utilizar

A imagem foi adquirida no OpenAerials https://openaerialmap.org  e posteriormente foi realizado o uploado parao github.

In [ ]:
##Acessar a imagem
raster_url = (
    "https://github.com/ambgeo/Youtube/raw/refs/heads/main/area_rio.tif"
)

In [ ]:
##Crie a pasta para baixar a imagem url
raster_path = geoai.download_file(raster_url)

File already exists: area_rio.tif


## Visualize a imagem baixada diretamente da Url

In [ ]:
geoai.view_raster(raster_url)

## Inicializeo  modelo.

In [ ]:
detector = geoai.CarDetector()

Model path not specified, downloading from Hugging Face...
Model downloaded to: /root/.cache/huggingface/hub/models--giswqs--geoai/snapshots/75788bf6253e967a91b9576b1620949a3b939820/car_detection_usa.pth
Model loaded successfully


## Extrair os carros.

* 1) Veja se os parametros estão ajustados a sua realidade.
* 2) Após ajustar os parâmetros cria a máscara.

Extrai a mascara da imagem que você subiu

In [ ]:
mask_path = detector.generate_masks(
    raster_path=raster_path,
    output_path="cars_masks.tif",
    confidence_threshold=0.5,
    mask_threshold=0.5,
    overlap=0.25,
    chip_size=(200, 200),
)

Dataset initialized with 30 rows and 43 columns of chips
Image dimensions: 6319 x 4353 pixels
Chip size: 200 x 200 pixels
Overlap: 25.0% (stride_x=150, stride_y=150)
CRS: EPSG:31983
Processing raster with 323 batches


100%|██████████| 323/323 [01:57<00:00,  2.75it/s]


Masks with confidence values saved to cars_masks.tif


Converta as mascara para GeoJson.

In [ ]:
gdf = detector.vectorize_masks(
    masks_path="cars_masks.tif",
    output_path="cars.geojson",
    min_object_area=50,
    max_object_area=2000,
)

Processing masks from: cars_masks.tif
Found 447 connected components
Using single-threaded processing, you can speed up processing by setting n_workers > 1


Processing components: 100%|██████████| 447/447 [00:14<00:00, 30.35it/s]

Saved 429 objects with confidence to cars.geojson
Total processing time: 15.04 seconds


## Adicione propriedades ao seu gdf

In [ ]:
gdf = geoai.add_geometric_properties(gdf)

## Visualize os resultados inicias e se precisar rode o modelo novamente

In [ ]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Utilize a proporção de área das propriedades dos gdf e filtre.

In [ ]:
gdf_filter = gdf[
    (gdf["area_m2"] > 4) & (gdf["area_m2"] < 60) & (gdf["minor_length_m"] > 1)
]

In [ ]:
len(gdf_filter)

411

## Visualize os resultados finais

In [ ]:
geoai.view_vector_interactive(gdf_filter, column="confidence", tiles=raster_url)

In [ ]:
geoai.view_vector_interactive(gdf_filter, tiles=raster_url)